In [120]:
import pandas as pd
import numpy as np
import pickle as pkl


from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegressionCV,LogisticRegression

In [121]:

data = pkl.load(open("datos_grupos/attrition_available_2.pkl",'rb'))

Instancias y num atributos

In [122]:
print("num de instancias y atributos:", data.shape)
print("Nombre de los atributos:", data.columns)

num de instancias y atributos: (4410, 31)
Nombre de los atributos: Index(['hrs', 'absences', 'JobInvolvement', 'PerformanceRating',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
       'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeCount', 'EmployeeID', 'Gender',
       'JobLevel', 'JobRole', 'MaritalStatus', 'MonthlyIncome',
       'NumCompaniesWorked', 'Over18', 'PercentSalaryHike', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')


tipo de datos

In [123]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 1 to 4409
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      3639 non-null   float64
 1   absences                 3575 non-null   float64
 2   JobInvolvement           3585 non-null   float64
 3   PerformanceRating        3534 non-null   float64
 4   EnvironmentSatisfaction  3428 non-null   float64
 5   JobSatisfaction          3637 non-null   float64
 6   WorkLifeBalance          3620 non-null   float64
 7   Age                      3636 non-null   float64
 8   Attrition                4410 non-null   object 
 9   BusinessTravel           3687 non-null   object 
 10  Department               3575 non-null   object 
 11  DistanceFromHome         3681 non-null   float64
 12  Education                3628 non-null   float64
 13  EducationField           4410 non-null   object 
 14  EmployeeCount           

Observamos missing values en la mayoria de atributos; la variavle de salida se corresponde con el índice 8 (Attrition) 

Miramos las columnas una a una; buscamos el tipo del atributo, si es constante (es decir si aporta información) y su proporción de missing values

In [124]:
info = {}
for colum in data:
    print(data[colum].value_counts(dropna = False, normalize = True ).to_frame())
    info[colum] = data[colum].value_counts(dropna = False, normalize = True ).to_frame()


               hrs
NaN       0.174830
6.033902  0.000454
9.853332  0.000454
6.002747  0.000227
5.691867  0.000227
...            ...
6.511941  0.000227
8.338820  0.000227
6.623272  0.000227
6.884605  0.000227
6.511790  0.000227

[3638 rows x 1 columns]
      absences
NaN   0.189342
7.0   0.049206
17.0  0.047846
6.0   0.045805
14.0  0.045578
19.0  0.045578
10.0  0.044671
8.0   0.044218
11.0  0.043991
18.0  0.043991
15.0  0.043991
12.0  0.043311
16.0  0.041270
13.0  0.040590
9.0   0.039683
20.0  0.039229
5.0   0.038095
21.0  0.032653
4.0   0.026304
22.0  0.020635
3.0   0.013605
23.0  0.011111
2.0   0.006576
24.0  0.001587
1.0   0.001134
     JobInvolvement
3.0        0.478912
2.0        0.208163
NaN        0.187075
4.0        0.078912
1.0        0.046939
     PerformanceRating
3.0           0.678458
NaN           0.198639
4.0           0.122902
     EnvironmentSatisfaction
3.0                 0.237415
4.0                 0.236508
NaN                 0.222676
2.0                 0.153515


In [125]:

for colum in data:
    if info[colum].shape[0] < 3:
        print(info[colum])


     Attrition
No    0.838776
Yes   0.161224
     EmployeeCount
1.0       0.790023
NaN       0.209977
       Over18
Y    0.805215
NaN  0.194785
     StandardHours
8.0        0.79161
NaN        0.20839


Consideramos que si una columna solo tiene dos valores (un valor y NaN) esta es constante; en este caso las columnas de Over18 y Standard Hours. Ignoramos la columna de attrition ya que es nuestra variable objetivo y se trata de una clasificación binaria.

Como ya se ha mencionado se trata de un problema de clasificación binaria. Y está desbalanceado (un 83% de la variable de salida se corresponde con una de las clases) 

Dividimos los datos en train y test

In [126]:
X = data.drop("Attrition", axis= 'columns')
Y = data['Attrition']

cat_col = []
num_col = []

for col in X.columns:
    if X[col].dtype != "object":
        num_col.append(col)
        continue
    cat_col.append(col)

cv = StratifiedKFold(n_splits=5)


n_fold = 1

for train, test in cv.split(X,Y):


    train_x = X.iloc[train]
    train_y = Y[train]
    
    test_x = X.iloc[test]
    test_y = Y[test]


    val_train = train_y.value_counts(dropna = False, normalize = True ).to_frame()
    val_test = test_y.value_counts(dropna = False, normalize = True ).to_frame()
    print(n_fold,val_train)
    print(n_fold,val_test)
    n_fold += 1


1      Attrition
No    0.836168
Yes   0.163832
1      Attrition
No    0.849206
Yes   0.150794
2      Attrition
No     0.84042
Yes    0.15958
2      Attrition
No      0.8322
Yes     0.1678
3      Attrition
No    0.837302
Yes   0.162698
3      Attrition
No    0.844671
Yes   0.155329
4      Attrition
No    0.842971
Yes   0.157029
4      Attrition
No    0.821995
Yes   0.178005
5      Attrition
No    0.837018
Yes   0.162982
5      Attrition
No    0.845805
Yes   0.154195


In [127]:

imputer_num = SimpleImputer(strategy='median')
scaler = RobustScaler()
pipeline_num = Pipeline(
    steps=[
        ("imputer", imputer_num),
        ("scaler", scaler)
    ]
)

imputer_cat = SimpleImputer(strategy='most_frequent')
encoder_cat = OneHotEncoder(drop = 'first')
pipeline_cat = Pipeline(
    steps=[
        ("imputer", imputer_cat),
        ("encoder", encoder_cat)
    ]
)

Log_reg = LogisticRegressionCV(class_weight='balanced', random_state=2, cv = cv, verbose= 4)

processor = ColumnTransformer(
    transformers=[
        ("num", pipeline_num, num_col),
        ("cat", pipeline_cat, cat_col),
    ]
)

predictor = Pipeline(
    steps=[
        ("Transformer", processor),
        ("predictor", Log_reg)
    ]
)



In [128]:

from sklearn.metrics import f1_score, balanced_accuracy_score, classification_report

Log_reg = LogisticRegression(class_weight='balanced', random_state=2)


n_fold = 1
for train, test in cv.split(X,Y):

    train_x = X.iloc[train]
    train_y = Y[train]
    
    test_x = X.iloc[test]
    test_y = Y[test]


    train_x_proc = processor.fit_transform(train_x)
    test_x_proc = processor.transform(test_x)

    Log_reg.fit(train_x_proc, train_y)
    pred = Log_reg.predict(test_x_proc)

    #f1 = f1_score(test_y, pred)
    #bal = balanced_accuracy_score(test_y, pred)
    
    report = classification_report(test_y,pred)

    print("\nfold:", n_fold)
    print(report)
    n_fold += 1












fold: 1
              precision    recall  f1-score   support

          No       0.87      0.53      0.66       749
         Yes       0.17      0.56      0.27       133

    accuracy                           0.54       882
   macro avg       0.52      0.55      0.46       882
weighted avg       0.77      0.54      0.60       882


fold: 2
              precision    recall  f1-score   support

          No       0.83      0.57      0.68       734
         Yes       0.16      0.42      0.24       148

    accuracy                           0.54       882
   macro avg       0.50      0.49      0.46       882
weighted avg       0.72      0.54      0.60       882


fold: 3
              precision    recall  f1-score   support

          No       0.85      0.55      0.67       745
         Yes       0.17      0.49      0.25       137

    accuracy                           0.54       882
   macro avg       0.51      0.52      0.46       882
weighted avg       0.75      0.54      0.60    

In [129]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score, balanced_accuracy_score, classification_report

predictor.fit(X,Y)

pred = predictor.predict()


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           44     M =           10

 L =  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00

X0 =  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.

 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           44     M =           10

 L =  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00  0.0000D+00
      0.0000D+00  0.0000D+00

X0 =  3.9140D-01  2.1834D-02  4.2150D-03  7.8748D-02 -2.0287D-01 -2.6756D-01
     -1.8099D-01 -2.1164D-01 -2.9816D-02 -5.5841D-02  0.0000D+00 -9.4095D-03
     -3.4397D-02 -5.2997D-02  1.4703D-01  2.2298D-02  0.0000D+00  4.7066D-03
     -2.5444D-01 -1.2172D-01 -7.2928D-02  1.5420D-01 -3.2428D-01  2.4371D-01
     -1.

 This problem is unconstrained.


...

 LINE SEARCH           0  times; norm of step =    5.0848328150186672E-003

At iterate   11    f=  1.94742D+03    |proj g|=  2.36057D-01

 X =  4.6869D-01  4.9714D-02 -4.7552D-03  2.4138D-01 -3.1959D-01 -6.1900D-01
     -3.1205D-01 -2.8557D-01 -2.7520D-02 -1.0215D-01  0.0000D+00 -2.9540D-03
     -5.8680D-02 -6.8313D-02  2.1749D-01  3.8633D-02  0.0000D+00  9.2914D-02
     -3.0000D-01 -1.4827D-01 -4.5426D-02  2.9607D-01 -6.3957D-01  1.5543D+00
      7.5829D-01 -5.8389D-01 -5.8329D-01 -1.1087D+00 -1.3550D+00 -1.1486D+00
     -1.3806D+00 -1.3876D+00  3.6234D-02  3.7489D-01  2.9798D-01  2.5334D-01
     -2.0801D-01  7.3201D-01  2.8374D-01  2.5136D-01  3.1108D-01  5.0846D-01
      1.1679D+00 -6.8781D-01

 G =  1.4764D-01  2.7239D-02 -5.1792D-03 -4.4989D-03  3.9531D-02  9.6541D-02
      2.0765D-04 -9.7565D-02  7.3089D-03 -3.8703D-02  0.0000D+00 -1.4386D-02
     -2.2645D-03  1.2160D-01 -2.3606D-01 -2.8825D-02  0.0000D+00  3.5264D-02
     -1.1411D-01  3.7094D-02 -5.0412D-02  2.0183D-01  1.0228D-

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


 LINE SEARCH           1  times; norm of step =    5.3750544176036821E-002

At iterate    1    f=  1.94471D+03    |proj g|=  1.06459D+01

 X =  4.9472D-01  2.8689D-02  1.3506D-02  1.1342D-01 -3.3767D-01 -6.4526D-01
     -3.7583D-01 -2.7328D-01  3.6746D-02 -1.1853D-01  0.0000D+00 -1.0301D-02
      3.7813D-04 -3.9166D-02  2.2523D-01  4.3969D-02  0.0000D+00 -3.9425D-03
     -3.1305D-01 -1.5251D-01  5.1706D-02  2.6794D-01 -6.9177D-01  1.2365D+00
      5.6283D-01 -7.5356D-01 -7.0031D-01 -4.3461D-01 -6.9734D-01 -4.4198D-01
     -7.1649D-01 -8.2016D-01 -7.7883D-02  1.6470D-01  1.0391D-01  8.8197D-02
     -3.2311D-01  6.6546D-01  1.8623D-01  2.0530D-01 -4.9634D-02  4.8126D-01
      1.1876D+00 -8.4738D-01

 G =  7.9730D+00 -1.3352D+00 -2.8134D+00  1.3503D+00 -5.0774D+00 -1.3247D+00
     -2.6802D+00 -2.9433D+00  2.1861D+00 -1.4271D+00  0.0000D+00 -2.1708D+00
      6.9735D-01  2.5858D+00  3.9139D+00  3.6119D+00  0.0000D+00 -1.5348D+00
     -4.3402D+00 -2.8235D+00 -1.7200D+00  3.0310D+00 -2.0249D-

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.5s finished
 This problem is unconstrained.


TypeError: Pipeline.predict() missing 1 required positional argument: 'X'